In [2]:
import numpy as np
import pandas as pd
import os

from tensorflow import keras
from tensorflow.keras import layers
from matplotlib import pyplot as plt

# Data Loading

In [3]:
data_path = '../../../data/train_data/host'
dfs = {}
for file in os.listdir(data_path):
    print('Saving ' + file[:-4] + ' into dfs')
    dfs[file[:-4]] = pd.read_csv(data_path+'/'+file) 

Saving dcos_docker into dfs
Saving os_linux into dfs
Saving db_oracle_11g into dfs
Saving mw_redis into dfs
Saving dcos_container into dfs


# Data Preprocessing

# Model Parameters

In [4]:
def get_model(x_train):
    model = keras.Sequential(
        [
            layers.Input(shape=(x_train.shape[1], x_train.shape[2])),
            layers.Conv1D(
                filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
            ),
            layers.Dropout(rate=0.2),
            layers.Conv1D(
                filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
            ),
            layers.Conv1DTranspose(
                filters=16, kernel_size=7, padding="same", strides=2, activation="relu"
            ),
            layers.Dropout(rate=0.2),
            layers.Conv1DTranspose(
                filters=32, kernel_size=7, padding="same", strides=2, activation="relu"
            ),
            layers.Conv1DTranspose(filters=1, kernel_size=7, padding="same"),
        ]
    )
    model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001), loss="mse")
    return model

def train_model(model, x_train):
    history = model.fit(
        x_train,
        x_train,
        epochs=60,
        batch_size=128,
        validation_split=0.1,
        callbacks=[
            keras.callbacks.EarlyStopping(monitor="val_loss", patience=5, mode="min")
        ],
    )
    return history

# Training

In [16]:
def normalise(df):
    mean = df['value'].mean()
    std = df['value'].std()
    if std == 0:
        df['value'] = df['value'] - mean
        return df
    df['value'] = (df['value'] - mean) / std
    return df

def gen_train_seq(values, time_steps=288):
    output = []
    for i in range(len(values) - time_steps):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)


models = []
histories = []
df=dfs['os_linux']
for name in list(df['name'].unique()):
    df_n = df[df.name == name]

    if df_n['value'].mean() == 0:
        continue
    print(df_n['value'].mean(), df_n['value'].std())

    if df_n.shape[0] >30000:
        time_step = 288
    elif df_n.shape[0] > 5000:
        time_step = 144
    else:
        time_step = 12

    x_train=np.empty((1,time_step,1))

    for host in list(df_n['cmdb_id'].unique()):
        df_nh = df_n[df_n.cmdb_id==host][['value']]
        df_nh = normalise(df_nh)
        x_train = np.concatenate((x_train, gen_train_seq(df_nh.values, time_step)), axis=0)

    nan_in = np.isnan(x_train)
    x_train[nan_in] = 0

    print(x_train[0][0][0])
    if np.sum(x_train) < .1 and np.sum(x_train) > -.1:
        print(name, ' constant 0s, not building a model')
    else:
        print('Training ', name)
        model = get_model(x_train)
        history = train_model(model, x_train)

        model.save('../../../data/vae/' + str(key) + '_' + name)
        models.append(model)
        histories.append(history)


0.02131268193440449 0.0220447502825371
0.0
Training  Processor_load_5_min
Epoch 1/60
179/179 [==============================] - 6s 34ms/step - loss: 0.2197 - val_loss: 0.0400
Epoch 2/60
179/179 [==============================] - 6s 33ms/step - loss: 0.0634 - val_loss: 0.0277
Epoch 3/60
179/179 [==============================] - 6s 32ms/step - loss: 0.0488 - val_loss: 0.0211
Epoch 4/60
179/179 [==============================] - 6s 33ms/step - loss: 0.0405 - val_loss: 0.0235
Epoch 5/60
179/179 [==============================] - 6s 36ms/step - loss: 0.0345 - val_loss: 0.0308
Epoch 6/60
179/179 [==============================] - 6s 33ms/step - loss: 0.0304 - val_loss: 0.0365
Epoch 7/60
179/179 [==============================] - 6s 32ms/step - loss: 0.0278 - val_loss: 0.0352
Epoch 8/60
179/179 [==============================] - 6s 32ms/step - loss: 0.0254 - val_loss: 0.0406
INFO:tensorflow:Assets written to: ../../../data/vae/db_oracle_11g_Processor_load_5_min/assets
0.0003787998358534045 0

442/442 [==============================] - 14s 32ms/step - loss: 0.0162 - val_loss: 0.0046
Epoch 5/60
442/442 [==============================] - 14s 31ms/step - loss: 0.0140 - val_loss: 0.0049
Epoch 6/60
442/442 [==============================] - 14s 32ms/step - loss: 0.0126 - val_loss: 0.0050
Epoch 7/60
442/442 [==============================] - 14s 32ms/step - loss: 0.0117 - val_loss: 0.0062
INFO:tensorflow:Assets written to: ../../../data/vae/db_oracle_11g_Incoming_network_traffic/assets
0.021622474747474748 0.15183056697195013
0.0
Training  Zombie_Process
Epoch 1/60
23/23 [==============================] - 1s 24ms/step - loss: 0.1938 - val_loss: 0.6671
Epoch 2/60
23/23 [==============================] - 0s 19ms/step - loss: 0.1150 - val_loss: 0.1741
Epoch 3/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0453 - val_loss: 0.0832
Epoch 4/60
23/23 [==============================] - 0s 19ms/step - loss: 0.0296 - val_loss: 0.0583
Epoch 5/60
23/23 [====================

23/23 [==============================] - 0s 20ms/step - loss: 0.1154 - val_loss: 0.0428
Epoch 6/60
23/23 [==============================] - 0s 19ms/step - loss: 0.1042 - val_loss: 0.0375
Epoch 7/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0958 - val_loss: 0.0335
Epoch 8/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0906 - val_loss: 0.0296
Epoch 9/60
23/23 [==============================] - 0s 19ms/step - loss: 0.0832 - val_loss: 0.0281
Epoch 10/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0789 - val_loss: 0.0253
Epoch 11/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0755 - val_loss: 0.0226
Epoch 12/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0711 - val_loss: 0.0231
Epoch 13/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0680 - val_loss: 0.0211
Epoch 14/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0647 - val_loss: 0.0191
Epoch 15/60
23/2

/Users/baconbaker/anaconda3/envs/ml/lib/python3.8/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


179/179 [==============================] - 6s 31ms/step - loss: nan - val_loss: nan
Epoch 2/60
179/179 [==============================] - 6s 32ms/step - loss: nan - val_loss: nan
Epoch 3/60
179/179 [==============================] - 5s 31ms/step - loss: nan - val_loss: nan
Epoch 4/60
179/179 [==============================] - 6s 31ms/step - loss: nan - val_loss: nan
Epoch 5/60
179/179 [==============================] - 6s 31ms/step - loss: nan - val_loss: nan
INFO:tensorflow:Assets written to: ../../../data/vae/db_oracle_11g_Processor_load_15_min/assets
0.01781405833333333 0.46865586423310707
0.0
Training  Page_pi
Epoch 1/60
179/179 [==============================] - 6s 33ms/step - loss: nan - val_loss: nan
Epoch 2/60
179/179 [==============================] - 6s 32ms/step - loss: nan - val_loss: nan
Epoch 3/60
179/179 [==============================] - 6s 33ms/step - loss: nan - val_loss: nan
Epoch 4/60
179/179 [==============================] - 6s 32ms/step - loss: nan - val_loss: na

0.0
Training  CPU_user_time
Epoch 1/60
179/179 [==============================] - 6s 33ms/step - loss: 0.2894 - val_loss: 0.0239
Epoch 2/60
179/179 [==============================] - 6s 31ms/step - loss: 0.0953 - val_loss: 0.0191
Epoch 3/60
179/179 [==============================] - 6s 32ms/step - loss: 0.0784 - val_loss: 0.0198
Epoch 4/60
179/179 [==============================] - 6s 32ms/step - loss: 0.0660 - val_loss: 0.0303
Epoch 5/60
179/179 [==============================] - 6s 32ms/step - loss: 0.0565 - val_loss: 0.0342
Epoch 6/60
179/179 [==============================] - 6s 33ms/step - loss: 0.0501 - val_loss: 0.0386
Epoch 7/60
179/179 [==============================] - 6s 33ms/step - loss: 0.0459 - val_loss: 0.0400
INFO:tensorflow:Assets written to: ../../../data/vae/db_oracle_11g_CPU_user_time/assets
1.0480545471763627 27.204201093303094
0.0
Training  Disk_svctm
Epoch 1/60
179/179 [==============================] - 6s 32ms/step - loss: 0.2055 - val_loss: 0.0252
Epoch 2/60
17

179/179 [==============================] - 6s 32ms/step - loss: nan - val_loss: nan
Epoch 5/60
179/179 [==============================] - 5s 30ms/step - loss: nan - val_loss: nan
INFO:tensorflow:Assets written to: ../../../data/vae/db_oracle_11g_Disk_rd_kbs/assets
25.686795522095956 23.42823532661403
0.0
Training  Memory_available
Epoch 1/60
179/179 [==============================] - 6s 32ms/step - loss: nan - val_loss: nan
Epoch 2/60
179/179 [==============================] - 5s 30ms/step - loss: nan - val_loss: nan
Epoch 3/60
179/179 [==============================] - 6s 32ms/step - loss: nan - val_loss: nan
Epoch 4/60
179/179 [==============================] - 5s 30ms/step - loss: nan - val_loss: nan
Epoch 5/60
179/179 [==============================] - 6s 31ms/step - loss: nan - val_loss: nan
INFO:tensorflow:Assets written to: ../../../data/vae/db_oracle_11g_Memory_available/assets
10.990583126483587 10.092498753962944
0.0
Training  Memory_used
Epoch 1/60
179/179 [=================

23/23 [==============================] - 0s 18ms/step - loss: 0.0371 - val_loss: 0.0220
Epoch 14/60
23/23 [==============================] - 0s 17ms/step - loss: 0.0344 - val_loss: 0.0211
Epoch 15/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0332 - val_loss: 0.0221
Epoch 16/60
23/23 [==============================] - 0s 19ms/step - loss: 0.0317 - val_loss: 0.0194
Epoch 17/60
23/23 [==============================] - 0s 19ms/step - loss: 0.0310 - val_loss: 0.0224
Epoch 18/60
23/23 [==============================] - 0s 17ms/step - loss: 0.0296 - val_loss: 0.0230
Epoch 19/60
23/23 [==============================] - 0s 19ms/step - loss: 0.0285 - val_loss: 0.0204
Epoch 20/60
23/23 [==============================] - 0s 17ms/step - loss: 0.0279 - val_loss: 0.0210
Epoch 21/60
23/23 [==============================] - 0s 17ms/step - loss: 0.0270 - val_loss: 0.0203
INFO:tensorflow:Assets written to: ../../../data/vae/db_oracle_11g_Cache_used/assets
374.8710542929293 29373.485

Epoch 15/60
23/23 [==============================] - 0s 16ms/step - loss: 0.0335 - val_loss: 0.0148
Epoch 16/60
23/23 [==============================] - 0s 15ms/step - loss: 0.0319 - val_loss: 0.0145
Epoch 17/60
23/23 [==============================] - 0s 14ms/step - loss: 0.0306 - val_loss: 0.0126
Epoch 18/60
23/23 [==============================] - 0s 16ms/step - loss: 0.0294 - val_loss: 0.0137
Epoch 19/60
23/23 [==============================] - 0s 17ms/step - loss: 0.0281 - val_loss: 0.0151
Epoch 20/60
23/23 [==============================] - 0s 17ms/step - loss: 0.0272 - val_loss: 0.0144
Epoch 21/60
23/23 [==============================] - 0s 15ms/step - loss: 0.0261 - val_loss: 0.0146
Epoch 22/60
23/23 [==============================] - 0s 16ms/step - loss: 0.0253 - val_loss: 0.0132
INFO:tensorflow:Assets written to: ../../../data/vae/db_oracle_11g_FS_used_pct/assets
109.84816363636362 108.01236941873964
0.0
Training  FS_total_space
Epoch 1/60
23/23 [=============================

23/23 [==============================] - 0s 20ms/step - loss: 0.0387 - val_loss: 0.0153
Epoch 14/60
23/23 [==============================] - 0s 20ms/step - loss: 0.0368 - val_loss: 0.0151
Epoch 15/60
23/23 [==============================] - 0s 20ms/step - loss: 0.0351 - val_loss: 0.0137
Epoch 16/60
23/23 [==============================] - 0s 19ms/step - loss: 0.0336 - val_loss: 0.0148
Epoch 17/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0325 - val_loss: 0.0134
Epoch 18/60
23/23 [==============================] - 0s 17ms/step - loss: 0.0309 - val_loss: 0.0140
Epoch 19/60
23/23 [==============================] - 0s 19ms/step - loss: 0.0300 - val_loss: 0.0139
Epoch 20/60
23/23 [==============================] - 0s 18ms/step - loss: 0.0291 - val_loss: 0.0125
Epoch 21/60
23/23 [==============================] - 0s 17ms/step - loss: 0.0278 - val_loss: 0.0140
Epoch 22/60
23/23 [==============================] - 0s 16ms/step - loss: 0.0271 - val_loss: 0.0135
Epoch 23/60
